<a href="https://colab.research.google.com/github/Riturajkumari/RandomForest/blob/main/RandomForest_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
## RandomForest Regression Solve it Internal Assignment
# output : total_bill, independent feature: tip,sex,smoker, day, time, size


### RandomForest Regression with Pipeline And Hyperparameter Tunning

In [73]:
import seaborn as sns
df= sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [74]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [75]:
df.isnull().sum()

,0
total_bill,0
tip,0
sex,0
smoker,0
day,0
time,0
size,0


In [76]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [78]:
# independet feature and Dependent feature
x=df.drop(labels=['total_bill'],axis=1)
y=df[['total_bill']]


In [79]:
x.head()

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4


In [80]:
y.head()

,total_bill
0,16.99
1,10.34
2,21.01
3,23.68
4,24.59


In [81]:
# Catagorical Feature sex,smoker,day,time to numerical feature
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
x['time']=encoder.fit_transform(x['time'])





In [82]:
x['time'].unique()

array([0, 1])

In [83]:
# Catagorical Feature sex,smoker,day to numerical feature
categorical_cols = ['sex', 'smoker','day']
numerical_cols = ['total_bill', 'tip','size']


In [84]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [85]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## Madle Missing Values
from sklearn.preprocessing import StandardScaler ## Feature Scalling
from sklearn.preprocessing import OneHotEncoder ## catagorical to numerical
from sklearn.compose import ColumnTransformer

In [86]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

# Define categorical and numerical columns
categorical_cols = ['sex', 'smoker', 'day']
numerical_cols = ['tip', 'size']

# Preprocessing for numerical data: Imputation and Scaling
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
    ('scaler', StandardScaler())                  # Scale numerical data
])

# Preprocessing for categorical data: Imputation and One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
    ('onehot', OneHotEncoder(drop='first'))               # Encode categorical data
])

# Combine the transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Create the complete pipeline with preprocessing and the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [87]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Make predictions using the test set
y_pred = pipeline.predict(X_test)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE) - optional but commonly used
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Print the results
print(f"R-squared: {r2}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


R-squared: 0.5106163820888993
Mean Absolute Error (MAE): 4.6488085568513124
Mean Squared Error (MSE): 41.49412439078377
Root Mean Squared Error (RMSE): 6.441593311501726


- Step 1: Define the Hyperparameter Grid
You specify the parameters you want to tune. Common hyperparameters for a RandomForestRegressor include:

- n_estimators: The number of trees in the forest.
- max_depth: The maximum depth of each tree.
- min_samples_split: The minimum number of samples required to split an internal node.
- min_samples_leaf: The minimum number of samples required to be at a leaf node.
- max_features: The number of features to consider when looking for the best split.

Explanation:
- param_grid: This dictionary specifies the hyperparameters and the range of values to explore.

- RandomizedSearchCV: This tool randomly samples the parameter space, making it more efficient than GridSearchCV when dealing with large datasets or many hyperparameters.

- n_iter: Number of parameter settings sampled. A higher number means a more exhaustive search but also longer runtime.
- cv: Cross-validation strategy. Here, a 3-fold cross-validation is used.
- verbose: Controls the verbosity of the output.
- n_jobs: Determines the number of jobs to run in parallel. -1 uses all processors.
- random_search.fit: Fits the model to the training data and performs the search for the best parameters.

- best_params: The best combination of hyperparameters found during the search.

- best_score: The best cross-validation score achieved.



In [89]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid
param_grid = {
    'model__n_estimators': [100, 200, 300, 400, 500],
    'model__max_depth': [None, 10, 20, 30, 40, 50],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__max_features': ['auto', 'sqrt', 'log2']
}



In [90]:
# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=pipeline,
                                   param_distributions=param_grid,
                                   n_iter=100,  # Number of parameter settings that are sampled
                                   cv=3,  # 3-fold cross-validation
                                   verbose=2,
                                   random_state=42,
                                   n_jobs=-1)  # Use all available cores

# Fit the model
random_search.fit(X_train, y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
90 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 427, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['tip',
                                                                                'size']),
                                                                              ('cat',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('onehot',
                                                                                                OneHotEncoder(drop='first'))]),
                                                                               ['sex',
                                                                                'smoker',
                                                                                'day'])])),
                                             ('model',
                                              RandomForestRegressor(random_state=42))]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'model__max_depth': [None, 10, 20, 30,
                                                             40, 50],
                                        'model__max_features': ['auto', 'sqrt',
                                                                'log2'],
                                        'model__min_samples_leaf': [1, 2, 4],
                                        'model__min_samples_split': [2, 5, 10],
                                        'model__n_estimators': [100, 200, 300,
                                                                400, 500]},
                   random_state=42, verbose=2)

In [91]:

# Get the best parameters and the best score
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score}")


Best Parameters: {'model__n_estimators': 100, 'model__min_samples_split': 5, 'model__min_samples_leaf': 2, 'model__max_features': 'sqrt', 'model__max_depth': None}
Best Cross-Validation Score: 0.43874793912762494
